In [1]:
import networkx as nx
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
filePath = 'C:\\Users\\Chris\\Desktop\\SI671Data\\network.tsv'

network_df = pd.read_csv('C:/Users/Chris/Desktop/SI671Data/network.tsv', sep='\t', names=['node', 'edge'])
G = nx.from_pandas_edgelist(network_df, source='node', target='edge')

Try to do a DFS from a random node and see how big of a component we generate. 

In [2]:
#uncomment if need to make a new G_sub, working with one from file now
#from random import choice
#node_list = list(G.nodes())
#G_sub = nx.ego_graph(G=G, n=choice(node_list), radius=4)

G_sub = nx.read_edgelist('g_sub.edgelist')

In [3]:
len(G_sub.node())

114403

'786434', '2918035', '6553603'

In [49]:
#Theory - we can initially filter on jaccard similarity between neighbors.
#If A and B are good friends, by some similarity measure, A is more likely to add B's friends. 
#We don't want to examine potential missing triads if A and B are not very good friends!

A = '2918035'
B = '2040012'

set1 =  set([n for n in G_sub[A]])
set2 = set([n for n in G_sub[B]])

#intersection of their friends divided by total number of unique friends
jaccard_sim = len((set1 & set2))/((len(set1)+len(set2))-(len(set1&set2)))

jaccard_sim





0.08571428571428572

In [67]:
min_friends = min(len(G_sub["6553603"]), len(G_sub["2918035"]))
max_friends = max(len(G_sub["6553603"]), len(G_sub["2918035"]))

#if the person with the least friends shares all of their friends in common with the person, they will have the following jaccard sim
min_friends/((max_friends+min_friends)-min_friends)

0.6428571428571429

In [102]:
def CalculateGoodFriends(G, node_id, potential_threshold, actual_threshold):
    good_friends_list = []
    #how many friends does this person have? 
    friends = len(G[node_id])
    #for a friend of the node_id
    for neighbor in G[node_id]:
        #we want to evaluate max possible jaccard sim with node_id
        #if it's not possible for them to be above .5, they can't be that good of friends, so let's exclude them right away
        #first, count all of their friends (2nd degree connections)
        friends_of_friends = len(G[neighbor]) 
        #check who has more total friends!
        if friends > friends_of_friends:
            max_jac_sim = friends_of_friends/((friends_of_friends+friends)-friends_of_friends)
        else:
            max_jac_sim = friends/((friends_of_friends+friends)-friends)

        #if it's possible to have a jaccard sim over the threshold, actually calculate it
        if max_jac_sim > potential_threshold: 
            friends_set =  set([n for n in G[node_id]])
            friends_of_friends_set = set([n for n in G[neighbor]])

            jaccard_sim = len((friends_set & friends_of_friends_set))/((len(friends_set)+len(friends_of_friends_set))-(len(friends_set&friends_of_friends_set)))
            #if the actual jaccard_sim is greater than .1, let's keep them as candidates moving forward!
            #note that this filters out 90% of potential matches that we have to search! cool
            if jaccard_sim >= actual_threshold:
                good_friends_list.append((neighbor))
            
    return [good_friends_list, friends]
    

In [105]:
def Find_Good_Friends(G, potential_threshold, actual_threshold, max_iter): 
    good_friends = []
    total_possible_friends = 0
    
    import time
    i = 0
    start = time.time()
    for person in G.node():
        if i < max_iter:
            good_friend_results = CalculateGoodFriends(G, person, potential_threshold, actual_threshold)
            
            if good_friend_results is not None:
                #keep track of how many total possible friends there were so we can know about exclusions
                total_possible_friends += good_friend_results[1]

                #if they have any good friends, store the result
                if (len(good_friend_results[0]) > 0):
                    for friend in good_friend_results[0]:
                        good_friends.append((person, friend))
            i+=1
    return [good_friends,len(good_friends)/total_possible_friends, (time.time() - start)]

results = Find_Good_Friends(G_sub, .4, .1, len(G_sub.node))

print("We filtered out {:.2%} of connections to find {} actual good friends!".format((1-results[1]), len(results[0])))

We filtered out 80.70% of connections to find 374592 actual good friends!


In [85]:
results[0][0:10]

[]

In [106]:
%%time
#we now want to do the same thing to the first iteration of good friends we found
#check through all of their friends and see who are they really connected with. 

two_hops_to_check = []

for edge in results[0]: 
    #print("Source: " + edge[0] + " Dest: " + edge[1])
    #get all of their friends    
    their_good_friends = CalculateGoodFriends(G_sub, edge[1], .5, .3)
    
    #edge[0] is the original node that we are looking for new links for
    #so lets match them up to these potentials
    for potential in their_good_friends[0]:
        #we could potentially have duplicates - if two of your friends were friends with the same person.
        #lets leave duplicates in because we can group on them later and count - we should weight by this! if you have 5 friends with the same friend that you aren't friends with
        two_hops_to_check.append((edge[0], potential))
    

Wall time: 7min 54s


<h4> Evaluate and sort results </h4>

Weight the Jaccard Similarity scores by the # of simultaneous friend's recommending and then sort to the top 50k new edges. 

In [109]:
df_jacc_sim = pd.DataFrame.from_records(two_hops_to_check, columns=['source', 'dest'])
df_jacc_sim['jacc_coef'] = [i[2] for i in nx.jaccard_coefficient(G_sub, two_hops_to_check)]

In [110]:
df_jacc_sim = df_jacc_sim[df_jacc_sim['jacc_coef'] < 1]
df_jacc_sim = df_jacc_sim[df_jacc_sim['jacc_coef'] > 0]

In [ ]:
df_jacc_sim_group = df_jacc_sim.groupby(['source', 'dest']).agg({'dest' : 'count', 'jacc_coef' : 'max'})

In [117]:
df_jacc_sim_group['sim'] = df_jacc_sim_group['dest']*df_jacc_sim_group['jacc_coef']

In [ ]:
df_jacc_sim_group = df_jacc_sim_group[['sim']].reset_index().sort_values(by = 'sim', ascending=False).iloc[0:49999]


In [125]:
df_jacc_sim_group[['source', 'dest']].to_csv("link_predictions.txt", header=None, index=None, sep=' ', mode='a')

<h3> Generating Features </h3>

Didn't actually get to this section :(

In [42]:
df = pd.DataFrame(index=G_s.edges())

In [43]:
df['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G_s, df.index)]
df['jaccard'] = [i[2] for i in nx.jaccard_coefficient(G_s, df.index)]

In [46]:
df.sort_values(by='jaccard', ascending=False).head()

,,preferential attachment,jaccard
6572838,2837156,1,0.0
4663414,3039020,1,0.0
4819921,4992755,1,0.0
5256045,2395166,1,0.0
6248813,3543337,1,0.0


local - look at direct neighbors of node

global - look at 1st, 2nd, 3rd order connections and say what attributes they all have
maybe frequent itemset of attributes for global?  What attributes commonly go together in the global 
   -nodes directly around say it should have T8, T10, T13
   
in the global section, whats the likelihood of this showing up?
 